In [1]:
import geopandas as gpd
import pandas as pd
from shapely import geometry

from srai.constants import REGIONS_INDEX, WGS84_CRS
from srai.embedders import CountEmbedder
from srai.joiners import IntersectionJoiner
from srai.loaders.osm_loaders import OSMOnlineLoader, OSMPbfLoader
from srai.loaders.osm_loaders.filters import HEX2VEC_FILTER
from srai.regionizers import H3Regionizer, SlippyMapRegionizer
from srai.utils import geocode_to_region_gdf

/home/marcin/Documents/mgr/semestr_II/nlp/nlp_l2/nlp_venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:

def get_embedding_for_region(region_name: str) -> pd.DataFrame:
    bbox_gdf = geocode_to_region_gdf(region_name)
    regionizer = SlippyMapRegionizer(z=16)
    regions_gdf = regionizer.transform(bbox_gdf)
    loader = OSMPbfLoader()
    features_gdf = loader.load(bbox_gdf, tags=HEX2VEC_FILTER)
    joiner = IntersectionJoiner()
    joint_gdf = joiner.transform(regions_gdf, features_gdf)
    embedder = CountEmbedder()
    embedding = embedder.transform(regions_gdf, features_gdf, joint_gdf)
    return embedding

In [ ]:
from tqdm import tqdm


with open('data/cities_v3.txt', 'r') as cities_file:
    for city in cities_file:
        print(city)
        city = city.replace("\n", "")
        path = f"data/embeddings/{city}.pkl"
        df = get_embedding_for_region(city)
        df.to_pickle(path)

In [ ]:
pd.read_pickle('data/embeddings/Tirana, Albania.pkl')

,aeroway_hangar,aeroway_helipad,amenity_arts_centre,amenity_atm,amenity_bank,amenity_bar,amenity_bbq,amenity_bench,amenity_bicycle_parking,amenity_bicycle_rental,...,tourism_museum,tourism_picnic_site,tourism_viewpoint,water_lake,water_pond,water_reservoir,water_river,waterway_canal,waterway_river,waterway_stream
region_id,,,,,,,,,,,,,,,,,,,,,
"(36364, 24482)",0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(36365, 24482)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
"(36366, 24482)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(36364, 24483)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(36365, 24483)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(36377, 24499)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(36378, 24499)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(36379, 24499)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
folium_map = plot_regions(regions_gdf, tiles_style="CartoDB positron", colormap=["rgba(0,0,0,0)"])
df.reset_index().explore(m=folium_map, column=REGIONS_INDEX, cmap=px.colors.qualitative.Bold)